Esercizio 1 

CONDOMINIO (Codice, Via, Ncivico, Comune, NumPiani, Amministratore)  
APPARTAMENTO (Condominio, NroProg, Piano, Nvani, MQ)   
PERSONA (CF, Cognome, Nome, ComuneResidenza)   
PROPRIETÀ (CF, Condominio, NroApp, Percentuale, DataInizio)   
COMUNE (Nome, Via, NAbitanti)  



• Comune di CONDOMINIO e ComuneResidenza di PERSONA riferiscono entrambi Nome di COMUNE  
• Condominio di APPARTAMENTO riferisce Codice di CONDOMINIO  
• CF di PROPRIETÀ e Amministratore di CONDOMINIO riferiscono entrambi CF di PERSONA  
• (Condominio, NroApp) di PROPRIETÀ riferisce (Condominio, NroProg) di APPARTAMENTO  


Scrivere le interrogazioni SQL che permettono di determinare  
1.	Le persone di Roma che hanno (almeno) un appartamento a Pescara.  
2.	Gli appartamenti del condominio ‘C01’ che hanno un unico proprietario, con indicazione di tutte le informazioni sul loro proprietario.  
3.	I condomini in cui ci sono almeno 10 appartamenti ed almeno 20 proprietari distinti.   
4.	Per ogni città, i condomini con almeno 10 piani situati in tale città, mantenendo nel risultato anche le città per cui non presenti condomini con almeno 10 piani  
5.	(OPZIONALE) Trovare le persone che sono amministratore di almeno un condominio in tutti i comuni   


In [ ]:
-- 1 - Le persone di Roma che hanno (almeno) un appartamento a Pescara.
SELECT DISTINCT p.CF, p.Cognome, p.Nome
FROM PERSONA p
JOIN PROPRIETA prop ON p.CF = prop.CF
JOIN APPARTAMENTO a ON prop.Condominio = a.Condominio AND prop.NroApp = a.NroProg
JOIN CONDOMINIO c ON  a.Condominio = c.Codice
WHERE p.ComuneResidenza = 'Roma' AND c.Comune = 'Pescara';

In [ ]:
-- 2 - Gli appartamenti del condominio ‘C01’ che hanno un unico proprietario, 
--     con indicazione di tutte le informazioni sul loro proprietario.  
SELECT a.*, p.CF, p.Cognome, p.Nome
FROM APPARTAMENTO a 
JOIN PROPRIETA prop ON a.Condominio = prop.Condominio AND  a.NroProg = prop.NroApp
JOIN PERSONA p ON prop.CF = p.CF 
WHERE a.Condominio = 'C01'
GROUP BY a.Condominio, a.NroProg, p.CF, p.Cognome, p.Nome
HAVING COUNT(*) = 1;

In [ ]:
 -- 3 - I condomini in cui ci sono almeno 10 appartamenti ed almeno 20 proprietari distinti.  
SELECT c.Codice, c.Via, c.Ncivico, c.Comune 
FROM CONDOMINIO c 
JOIN APPARTAMENTO a ON c.Codice = a.Condominio
JOIN PROPRIETA prop ON c.Codice = prop.Condominio
GROUP BY c.Codice, c.Via, c.Ncivico, c.Comune
HAVING COUNT(DISTINCT a.NroProg) >= 10 AND COUNT(DISTINCT prop.CF) >= 20;

In [ ]:
SELECT cm.Nome AS Città, c.Codice AS Condominio
FROM COMUNE cm
LEFT JOIN CONDOMINIO c ON cm.Nome = c.Comune AND c.NumPiani >= 10;


In [ ]:
SELECT p.CF, p.Cognome, p.Nome
FROM PERSONA p
JOIN CONDOMINIO c ON p.CF = c.Amministratore
GROUP BY p.CF, p.Cognome, p.Nome
HAVING COUNT(DISTINCT c.Comune) = (SELECT COUNT(*) FROM COMUNE);


In [ ]:
-- Creazione  delle tabelle 
CREATE TABLE COMUNE (
    Nome VARCHAR(50) PRIMARY KEY,
    Via VARCHAR(100),
    NAbitanti INT
);

CREATE TABLE PERSONA (
    CF CHAR(16) PRIMARY KEY,
    Cognome VARCHAR(50),
    Nome VARCHAR(50),
    ComuneResidenza VARCHAR(50),
    FOREIGN KEY (ComuneResidenza) REFERENCES COMUNE(Nome)
);


CREATE TABLE CONDOMINIO (
    Codice CHAR(5) PRIMARY KEY,
    Via VARCHAR(100),
    Ncivico INT,
    Comune VARCHAR(50),
    NumPiani INT,
    Amministratore CHAR(16),
    FOREIGN KEY (Comune) REFERENCES COMUNE(Nome),
    FOREIGN KEY (Amministratore) REFERENCES PERSONA(CF)


);

CREATE TABLE APPARTAMENTO (
    Condominio CHAR(5),
    NroProg INT,
    Piano INT,
    Nvani INT,
    MQ INT,
    PRIMARY KEY (Condominio, NroProg),
    FOREIGN KEY (Condominio) REFERENCES CONDOMINIO(Codice)
);

CREATE TABLE PROPRIETA (
    CF CHAR(16),
    Condominio CHAR(5),
    NroApp INT,
    Percentuale DECIMAL(5,2),
    DataInizio DATE,
    PRIMARY KEY (CF, Condominio, NroApp),
    FOREIGN KEY (CF) REFERENCES PERSONA(CF),
    FOREIGN KEY (Condominio, NroApp) REFERENCES APPARTAMENTO(Condominio, NroProg)
);

In [ ]:
-- LEFT JOIN
SELECT a.targa, a.marca, a.modello, g.nome, g.cognome
FROM automobile a
LEFT JOIN Guidatore g 
ON a.guidatore = g.nroPatente;

In [ ]:
-- RIGHT JOIN
SELECT a.targa, a.marca, a.modello, g.nome, g.cognome
FROM Automobile a
RIGHT JOIN Guidatore g
ON a.guidatore = g.nroPatente;

In [ ]:
-- FULL OUTER JOIN
SELECT a.targa, a.marca, a.modello, g.nome, g.cognome
FROM Automobile a
FULL OUTER JOIN Guidatore g
ON a.guidatore = g.nroPatente;


In [ ]:
-- su mariadb il  full outer join non è supportato quindi  bisogna unire il left + il right.
SELECT a.targa, a.marca, a.modello, g.nome, g.cognome
FROM Automobile a
LEFT JOIN Guidatore g
ON a.guidatore = g.nroPatente
UNION
SELECT a.targa, a.marca, a.modello, g.nome, g.cognome
FROM Automobile a
RIGHT JOIN Guidatore g
ON a.guidatore = g.nroPatente;


In [ ]:
-- NTURAL JOIN 
SELECT a.targa, a.marca, a.modello, a.guidatore
FROM Automobile a
NATURAL JOIN Automobile1;

Esercitazione 2024-12 dal pdf. della prof:
https://fad.unich.it/pluginfile.php/347057/mod_resource/content/2/2024-12-XXEsecitazioneSQL.pdf.fv cgdr

In [1]:
create table AEROPORTO
(Città varchar(20) primary key,
Nazione varchar(20) not null,
NumPiste integer);

create table VOLO (IdVolo char(6),
GiornoSett enum('lunedì', 'martedì', 'mercoledì', 'giovedì', 'venerdì', 'sabato', 'domenica'),
CittàPart varchar(20) not null references AEROPORTO(Città),
OraPart time not null,
CittàArr varchar(20) not null references AEROPORTO(Città),
OraArr time not null,
TipoAereo char(20) references AEREO(TipoAereo),
primary key(IdVolo, GiornoSett));

create table AEREO (
TipoAereo char(20) primary key,
QtaMerci float,
NumPasseggeri smallint not null);

SyntaxError: invalid syntax (3881921383.py, line 1)

Scrivere in sql le interrogazioni che permettono di determinare:
1. Le città  con un  areporto di cui non è noto il numero di piste;
2. Le nazioni  da cui parte  e arriva il volo con codice AZ274;
3. I tipi i aereo usati nei voli che partono da Torino.
4. I  tipi  di aereo e il corrispondente numero di passeggeri per i tipi i aereo usati nei voli che partono da Torino.
5. Le città da cui prtono voli internazionali;
6. Le città da cui partono voli diretti a Bologna, ordinate alfabeicamente;
7. Il numero di voli internazionali che partono il giovedì da Napoli;
8. Il numeori di voli  internazionali che partono ogni settimana da città italane (farlo in due modi, facendo comparire o meno nel risultato gli aereoporti senza voli internazionli);
9. Le città francesi da cui partono più di 20 voli alla settimana iretti in italia.
10. Gli areoporti italiani che hanno solo voli interni. Rappresrntare questa interrogazione in più modi:
    - con operatori insiemistici 
    - con una intrrogazione nidificata con l'operatore not in;
11. Le città che sono  servite daal'aereo caratterizzato dal massimo numero di passeggeri.

In [ ]:
-- 1.  Le città  con un  areporto di cui non è noto il numero di piste;
select città 
from aereoporto
where NumPiste is null;

In [ ]:
-- 2. Le nazioni  da cui parte  e arriva il volo con codice AZ274;
--    Per identificare le nazioni di partenza e arrivo del volo AZ274, 
--    dobbiamo effettuare un join tra VOLO e AEROPORTO due volte (una per la città di partenza e una per quella di arrivo):
select distinct a1.nazione as nazionePartenza, a2.nazione as nazioneArrivo
from volo v
join aeroporto a1 on v.CittàPart = a1.città
join aeroporto a2 on v.CittàArr = a2.città
where v.IdVolo = 'AZ274';

In [ ]:
-- 3. I tipi i aereo usati nei voli che partono da Torino.
--    L’operatore DISTINCT viene usato per eliminare i duplicati dai risultati di una query.
select distinct v.TipoAereo
from volo v 
where v.CittàPart = 'Torino'; 

In [ ]:
-- 4. I tipi di aereo e il corrispondente numero di passeggeri per 
--    i tipi i aereo usati nei voli che partono da Torino.
select distinct a.TipoAereo, a.NumPasseggeri
from volo v 
JOIN aereo a on v.TipoAereo = a.TipoAereo
where v.CittàPart = 'Torino';

In [ ]:
-- 5. Le città da cui prtono voli internazionali;
select distinct a1.Città
from volo v 
join aeroporto a1 on v.CittàPart = a1.Città -- collegamento per la città di partenza 
join aeroporto a2 on v.CittàArr = a2.Città -- collegamento per la città di arrivo
where a1.Nazione <> a2.Nazione;

In [ ]:
-- 6. Le città da cui partono voli diretti a Bologna, ordinate alfabeicamente;
select distinct CittàPart
from volo
where CittàArriv = 'Bologna'
order by CittàPart;

In [ ]:
-- 7. Il numero di voli internazionali che partono il giovedì da Napoli;
select count(*) as NumVoliInternazionali
from volo v 
join aeroporto a1 on v.CittàPart = a1.Città -- collegamento per la città di partenza 
join aeroporto a2 on v.CittàArr = a2.Città 
where a1.Città = 'Napoli'
    and v.GiornoSett = 'giovedì'
    and a1.Nazione <> a2.Nazione;

In [ ]:
select count(*) as NumVoliInternazionali
from volo v, aeroporto a1,  aeroporto a2
where a1.Città = v.cittàPart and v.CittàArr = a2.Città  -- le condizioni di join naturale esplicite nel where
    and a1.Città = 'Napoli'
    and v.GiornoSett = 'giovedì'
    and a1.Nazione <> a2.Nazione;
 -- invece join

In [ ]:
-- 8.a Il numeori di voli  internazionali che partono ogni settimana da città italane 
-- non facendo comparire nel risultato gli aereoporti senza voli internazionli;
SELECT A1.Città, COUNT(*) AS NumVoliInternazionali
FROM VOLO V, AEROPORTO A1,  AEROPORTO A2 
JOIN AEROPORTO A1 ON V.CittàPart = a1.Città
JOIN AEROPORTO A2 ON V.CittàArr = a2.Città
WHERE A1.Nazione = 'Italia'
    AND A1.Nazione <> A2.Nazione
GROUP BY A1.Città;

In [ ]:
-- 8b. Il numeori di voli  internazionali che partono ogni settimana da città italane 
-- facendo comparire nel risultato gli aereoporti senza voli internazionli;
SELECT A1.Città, COUNT(V.IdVolo) AS NumVoliInternazionali
FROM AEROPORTO A1
JOIN VOLO V ON V.CittàPart = a1.Città
JOIN AEROPORTO A2 ON V.CittàArr = a2.Città
WHERE A1.Nazione = 'Italia'
    AND (A1.Nazione IS NULL OR A1.Nazione <> A2.Nazione)
GROUP BY A1.Città;

In [ ]:
-- Per vedere i voli associati a ogni città italiana, puoi utilizzare una query con JOIN per collegare la tabella AEROPORTO con VOLO
select a.Città as CittàPartenza, v.IdVolo, v.GiornoSett, v.CittàArr, v.OraPart, v.OraArr
from aeroporto a
join volo v on a.Città = v.CittàPart
where a.nazione='Italia';

In [ ]:
-- 9. Le città francesi da cui partono più di 20 voli alla settimana diretti in italia
SELECT A1.Città, COUNT(*) AS NumVoli
FROM VOLO V
JOIN AEROPORTO A1 ON V.CittàPart = A1.Città
JOIN AEROPORTO A2 ON V.CittàArr = A1.Città
WHERE A1.Nazione = 'Frnacia'
    AND A2.Nazione = 'Italia'
GROUP BY A1.Città
HAVING COUNT(*) > 20;

In [ ]:
-- 10a. Gli areoporti italiani che hanno solo voli interni. Rappresrntare questa interrogazione in più modi:
    -- con operatori insiemistici;

-- insieme degli aeroporti italiani
SELECT Città 
FROM AEROPORTO
WHERE Nazione = 'Italia'
EXCEPT
-- insieme degli aeroporti italiani con voli internazionali ADD
SELECT DISTINCT a.Città
FROM AEROPORTO a 
JOIN VOLO V ON a.Città = v.CittàPart
WHERE a.Nazione = 'Italia' AND v.CittàArr NOT IN (SELECT Città FROM AEROPORTO WHERE Nazione = 'Italia');

In [ ]:
-- 10b. Gli areoporti italiani che hanno solo voli interni. Rappresrntare questa interrogazione in più modi:
    -- con una intrrogazione nidificata con l'operatore not in;

SELECT a.Città
FROM AEROPORTO a
WHERE a.Nazione = 'Italia'
AND a.Città NOT IN (
    SELECT DISTINCT v.CittàPart
    FROM VOLO v
    WHERE v.CittàArr NOT IN (SELECT Città FROM AEROPORTO WHERE Nazione = 'Italia')
);

In [ ]:
-- 11. Le città che sono  servite dall'aereo caratterizzato dal massimo numero di passeggeri.
SELECT TipoAereo
FROM AEREO
WHERE NumPasseggeri = (SELECT MAX(NumPasseggeri) FROM AEREO);

In [ ]:
SELECT DISTINCT a.Città
FROM AEROPORTO a
JOIN VOLO v ON a.Città = v.CittàPart OR a.Città = v.CittàArr
WHERE v.TipoAereo = (
    SELECT TipoAereo
    FROM AEREO
    WHERE NumPasseggeri = (SELECT MAX(NumPasseggeri) FROM AEREO)
);

Eserci